<img src="imagenes/rn3.png" width="200">
<img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="200">

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Redes recurrentes, implementación simple

[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 9 de mayo de 2018.

En esta libreta vamos a explorar como desarrollar redes recurrentes, simples y basadas en unidades de memoria de largo y corto plazo (LSTM), aplicadas a la generación automática de texto.

Dado que estamos en México en año electoral, y que se vota por diputados y presidentes municipales en muchisimos municipios del país, nos preguntamos si podríamos inventar nuevos municipios para que todos los candidatos tuvieran un lugar que gobernar. Así, generamos una lista con el nombre de todos los municipios de México, y la vamos a usar para aprender los nombres, y generar nombres a partir de una red recurrente. Esto es interesante ya que en México hay muchos municipios cuyos nombres tienen raices del español, el nahuatl, direrentes lenguas mayas, varias lenguas de la familia yuto-azteca, e inclusive algunos que son palabras inventadas (como Mexicali). Así que generar nombres de municipios mexicanos *creibles* es un problema interesante.

El archivo con el nombre de los municipios se encuentra para su descarga [aqui](municipios.txt).


## 1. Redes recurrentes: Desarrollar una red recurrente completamente a pie.

Con el fin de entender como funcionan las redes neuronales, vamos a aplicar un modelo de generación de texto *letra a letra*, en el cual tanto la arquitectura como el método de aprendizaje sean programados a mano. 

No vamos a pedir que lo programen todo solos, simplemente que utilicen el modelo programados en este [gist](https://gist.github.com/karpathy/d4dee566867f8291f086), y lo adapten a leer el nombre de los municipios y a generar nombres de municipios.

Para esto:

1. Copiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Copiar y comentar en español el contenido del método de verificción de gradiente (para limpiar el código de errores) y usarlo por unas cuantas iteraciones para demostrar que el algoritmo de entrenamiento funciona correctamente.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


### Librerias

In [13]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np
from random import uniform

### Cargar datos

In [2]:
# data I/O
data = open('municipios.txt', 'r').read() # Lee el archivo que contiene las palabras a aprender
chars = list(set(data)) # Obtiene los diferentes caracteres usados
data_size, vocab_size = len(data), len(chars) # Tamaño del dataset y del vocabulario (diferentes letras que usan los municipios)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 35530 characters, 68 unique.


### Función de perdida

In [10]:
def lossFun(inputs, targets, hprev):
    """
    Funcion de perdida
    
    Parámetros
    ----------
    inputs: número de entradas de la red
    targets: número de 
    hprev: 
    
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

### Muestra de enteros del modelo

In [8]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

### Verificador de gradiente


In [ ]:
# gradient checking
def gradCheck(inputs, target, hprev):
    global Wxh, Whh, Why, bh, by
    num_checks, delta = 10, 1e-5
    _, dWxh, dWhh, dWhy, dbh, dby, _ = lossFun(inputs, targets, hprev)
    for param,dparam,name in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], ['Wxh', 'Whh', 'Why', 'bh', 'by']):
        s0 = dparam.shape
        s1 = param.shape
        assert s0 == s1, 'Error dims dont match: %s and %s.' % (`s0`, `s1`)
        print(name)
        for i in range(num_checks):
            ri = int(uniform(0,param.size))
            # evaluate cost at [x + delta] and [x - delta]
            old_val = param.flat[ri]
            param.flat[ri] = old_val + delta
            cg0, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)
            param.flat[ri] = old_val - delta
            cg1, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)
            param.flat[ri] = old_val # reset old value for this parameter
            # fetch both numerical and analytic gradient
            grad_analytic = dparam.flat[ri]
            grad_numerical = (cg0 - cg1) / ( 2 * delta )
            rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
            print('%f, %f => %e ' % (grad_numerical, grad_analytic, rel_error))
            # rel_error should be on order of 1e-7 or less

### Inicializar los parámetros del modelo

In [50]:
tol = 17

# hyperparameters
hidden_size = 38 # size of hidden layer of neurons
seq_length = 12 # number of steps to unroll the RNN for
learning_rate = 0.2

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

### Main

In [51]:
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % hidden_size == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % hidden_size == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 
    if smooth_loss < tol:
        break

----
 ÑüdYOohiGifgn
ZcOÁé
is DFZ-aLzPqAL
süázq.JÁ6JHJKGálvmhltRvxNVkjiMKñnL-
büOcSfAHÑzOhEZIáCRH8NCH-vtnJphSv-MÑLyIfxdláutHfsm8RNK
ii0kVPóSÑñbmFKbÑ-nY0dL.ZVD00MábHvvTgoJüR8PíNNqDdUéG.6dMTZHélf0JJEax,jCaUÁHG 
----
iter 0, loss: 50.634093
----
 cuaAa

abuu
A
AGulaeuaua
Acua
uu

c
A
u
Aguluuu n
An
AcurzcuAAAulaeueuaaaaul
luuxalu
leuuAlAAaAealuPA8uluubue
lusAu
Tuesbuu
AaualAyauupAbu
AguuPlPuuu
aAbauAuueeuAeAuA
A
scu
luAluPuabuu
auuuuuA
a uuAue 
----
iter 38, loss: 50.741284
----
  Lala ia LtiArooLAlLe2tapectoaLíettucormctcLanehLtoeLloCrephtrdiLoiaatrtorLmcu uouaoaátue LPaAhDKtTLatidcoAoLddruoLeaoel  RAlostoA
olaiRetyepiiLooacelPeomcocoérrmAl. oL
AtlitrLdrr coe LopLetLttacLc y  
----
iter 76, loss: 50.321356
----
 zaA
Anazzol GiuaGdnnaaAÁjpápoa
ppzpxao
AgaGiÁlesoáhAng
ApáplgloClanlalaxenpixae
Apaxemaiacitlenaa 
Alap
Anxan
zn
AnlrApaooAep n
Angpeieclintersan eeanllxoeAhacetipzJx
ApGe
AprnzAn
Alg ngtsá
Ap
Aniiaed 
----
iter 114, loss: 49.791548
----
 
Aren   ecsBllíeno o non
Anan oenuie B
Ama

----
 oro
Sucbas S
mamas
O Gucarlandocoa
Santigaa
Azosa
Saálaí
SoAliya
Saero
Saydran
Sarisahuórenachtigo
Rua
Sixrelallerisderurana
Ririlanto Siccamiyanando
Ríariíro
SuGyirtan
Sueres
Sariqamos
Pariogo
Sazeri 
----
iter 1330, loss: 34.121338
----
 de Aan Sumémen Ovis Atda Sal
A Dmiri AgCriutas T. Acb xui Cuiro
Fen Iadatltíra de Sin
San Am AsEczos dé Anubranta
Sas Axanla
San
Shuusinacle
Salo
Sun Sén Cuisitis Anuagas Buess
Sanmo
Sontirífan Atacan 
----
iter 1368, loss: 33.853612
----
 AnSin  a
Ainos Tal
Sage Ca
San
Cona de ua
Sinin Antén Caoto  otatipe
Sati
Sonto Bán Sénmo Cexa Anlo
San Anbirita Aztan Acdaquñid
Baros Cas da Can
San  Schan
Sanio Sénto Nál Calio Aidito Co Anto
las Bo 
----
iter 1406, loss: 33.398635
----
 jo Ronlotenasley Bechomaxhocan
Sox
Say
Bahuelán Alan
San
Sal Bainanen
SantamorSa
San Bránto Acarco
San
San Bakuoman
San Qmumtl BeHtSue
San BhurQ Yocuemo Slpio San
Sen Bidzacho
Sártonepe
San ouíupec
So 
----
iter 1444, loss: 33.030902
----
 c
San Fegua
Alto Mac
Isdepeóren a


----
 zangotúcoan
Tllancollanco
Tlacana
Tlaxuas
Taxco
Tlanánca Calida
Tla
Taxiquiguucac
Mamo
Tlaqua
Tlaltajda
Tamla
Tayitláellantaxcag
Tlalo
Tlarío
Tlacé
Tlalaco de o
Tlasola
TlaMezban
Tlanamama
Tepinanpatl 
----
iter 2698, loss: 24.168162
----
 mcilás de
Tue Peyuivascla
oucumuncom
Tonlquililtlíutechel
Tantiman
Tlache Julu
Tonde Zontlru
Tgnec
Temcuquilám
Zulu
Tlo dem mumen
Teméncolimento
Tunin de ío
Tojulrez
Totlayelc
Tamula
Totla Bultatlalát 
----
iter 2736, loss: 24.173262
----
 appa Vaza delmac
Trilá
UrTlón
Vridal RepDípa Varlepa
Vogellla
Valápohzila de Zleé Vaprlia de da Allpreltla de de Velú
Vilanláe dC -vaz
kádetlo de Morepid
lalo
Tulloco
Vriónhuila
de Vllríz de Gueledroo 
----
iter 2774, loss: 24.270822
----
 gocuale de Cla Vila
Volacavio di Calapa de Cazitlca de Gullan de Gullas de Ela
Vilcangui
Vilto
Villa ds Cola dr
Suo
Villa Viatla
VillaVa
Vela Ala di de Miszuz de Alatapacanga
Valaca Guilyalde la
Vilda 
----
iter 2812, loss: 24.224637
----
  Tetiópe
Villa Vilóntin
Villa Dlal

----
 orehumreco
Maviruc
Mixcorutixtorato
Miltes
Momrehua
Mantlas
Mixtiforoxterenenaro del Pránes
Mantoscoromlos
Mostepol
Muxcos de Merrron
Megxnoro
Mexcay Tocán Cos
Mostlátrolaran
Máncoxtonlirgocrete Morte 
----
iter 4066, loss: 24.470343
----
 ebú
Mipáz
Nagra Ne Notopes
NaatjaVitepas
Navila Nape
Navhuacan de Hrelo
Notoroz
Nortovil del Peo
Nata
Nauca Natja de
Nquapa
Nantas de dezo
Navaxtla
Narasca
Nixania
Náraz
Naman
Naoxilla
Nuatlen
Naciazl 
----
iter 4104, loss: 24.501635
----
 uáno
Ocono
Molfo
Ocoso
Mocez
Oeganco Bv Ocoscoco
Oáuéc
Ocofcán
Ocogo
Atontác
O
Miramos
Juarcacil Oto
Fistobelgo
Ocotez
Opodro
Momho
Oacapán
Opunmo
Jomavo
Dorarua
Ovuápepen Delala
Ococpam
Mechacame
Oco 
----
iter 4142, loss: 24.514252
----
 ltaculmolpa
OcaPlella
Pilamac
Pecra Capanca
Dovutolunda
Padan Rlatlán
Pamuna
Palutava
Pantatelilas
Páncava Atuvamapen de Juapca
Nuarixalo
Mahuá
Igoczigucasla de Pante Párejacitlaz Rayacá
Pimaro
Baayar 
----
iter 4180, loss: 24.574303
----
 cutaño
Petmaban de Ball
Pelapa
Pen

----
 a
Tutepa
Tachialolánlá
Tacópalahuarpralalan
Tamadá
Tacantaroaro
Tamaapatlán
Morazanda
Tanrolatlán
Taranaqultatatlapa
Tamrulalulca
Tatzila
Tasiláculpa
Raadachiala
Tabtaltepec
Talentla
Zanázanta
Juuaxiu 
----
iter 5434, loss: 20.590145
----
 emécapimodhicamana
Tamamastepetlán del Pecuvuca
Tecualcaepen de Morlaro
Talgnaquegquatatzispen
Techilanteá
Tepaalatla
Teruachiloz Asalallo
Tajotpas
Temcalquecamóc
Tecalhilorringuacola
Teca
Tucachuelpe 
----
iter 5472, loss: 20.709309
----
 e esampem
Teoyogo de Gualiptepichu
Tanlanca
Tenán
Temakonpa
Tepeypepe
Temertepec
Temontepen de Egontaago Guixquepán
Tentoghíeco
Teestlpos
Turuchatepin
Tehingo cacó
Tepacla
Trá
Temtítzontióno
Tegocha
T 
----
iter 5510, loss: 20.726466
----
 Gejésrlca
Tepulac
Tetepatla
Tehédrtepan
Tepénco
Tapees
Teazalanto
Tepipuaco
Textla
Tepchepaz
Tepenmalldil
Tepatla
Tendele
TeNopachriz
Tlajotlaz
Tepizdec
Teozazoc
Tepego de Mapatlán
Tzdedzol
Tepiat
Tep 
----
iter 5548, loss: 20.699664
----
 ac
Tlala
Tatlán
Telantla
Tuetetla


...

----
 a
Tlapatlánco
Tlaulo
Tlahuaco
Sitacuilda
Tlalz del Tlavo Ocolteped
Temosan
Tlapan
Tla
Tlajolja
Terlillcilu
Tlajo Vilco
Siqula
Tlalá
Tlala
Tlapan
Tlihtánzalché
Tolacáca
Tlahuilla
Tlamuxtro
Tlihuiluill  
----
iter 32262, loss: 18.210760
----
 o
Tlachtahuilcán
Tlano da OnAmano
Tlapa
Tlaxilla
Tlapatlal
Tlabán
Elpenanilapa Alexual
Tlapecuel
Tamango
Tecodanar
Tepemónca
Tlahuacoxcaco
Tlapolguilulcaclin
Siltagpac
Toxtla Díanco
Bahocraltíca
Tlaro 
----
iter 32300, loss: 18.237427
----
 Trirría
Teolo de ula
Tumonasco de uatepec
Tuyitlám Ixcareta
Tuhuiu
Tratlár
Tuxtepen de Gorcapachima Villán
Trirapo
Tualblás de Tlandel
Tlbepán
Tlatlán
Trongonu
Tuulos
Tlayrero
Teltepec Ronuca
Tulinari 
----
iter 32338, loss: 18.332318
----
 
Vallpan de Doxtla de Guerilco
Ureo de Amila
Trteped
Tulhua
Tucicorsizo de Conitzacantil
Urenoma
Tuliaco
Tuló
Pualco Maranzis
Villaga di de de Hormava
Uranchizídaaltobán
Rzanteteárma de Zezcilgo Quiza 
----
iter 32376, loss: 18.520052
----
 xta Vión
Ríbala
Villpana
Mmaxt

----
 xicor
Manexmarepec
Maderesa
Juihtemetepes
Molína
Margo
Jalfla
Lalquinda
Macunalpan
Matitlis
Manta Magrero
Cos Malcos
Ilarantepei
Mapila
Mandancpa
Manazo de Marenal
Midaquea
Manoma Matíaé
Galilas
Maroa 
----
iter 33592, loss: 21.113440
----
 lte
Mecuatlán de Matí
Macaliadetlán Magubaxco de Bedecatez
Jecaras
Mía zaldomac
Meslares
Marequec
Méstoa de Zaha Fides
Méntel
Maderre
Miadilriotlall Micpatepec
Miadancil de ún Maxlo
Milapa
Mizavebé
Ma 
----
iter 33630, loss: 21.099738
----
 ontlahuilosco
Monenelánitlán
Melotlán
Rezezosgo de Pujargua de Becall
MRode
Multilzo
Mloratlán
Mirereno Monez
Mamyac
Nox
Montlticango
Morotla leroconaral Citzotláe
Miniteár
Inidrilartán de Mamtos Mang 
----
iter 33668, loss: 21.078746
----
 aguitlán
Nanorea
Nivazzo
Nuea
Nizameo Nercánsca
Maloro
Marón
Najalteáros de Ría Ronapinguapar
Jacotlal Náratedo
Nato de Moraco
Namehue
Narizango
Orteran
Namemamo de le del Juaniso
Náoconga
Naló
Namari 
----
iter 33706, loss: 21.204082
----
  MedateKapas Oes
Mévequiquigo


----
 laalos
Santoapa
Santa Mapinago Oluta Tolquinán
Santayamo
Sancinde
Santiago Ticaran Tequilco
Santiago Tetepe Juánigo Sieltilipec
Santiaguala
Santiago Ticanero
Santisos
Santiaguilu
Santiago Terongo Clal 
----
iter 34884, loss: 17.244162
----
 co
Santo Dua Quinzacánmá
Santo Ixcango Topigo Yucablo  Rsloltepe Rohoxcatlán
Santmago Zacío Tentepitáho
Santuatlán Andón Pilago Yunlánín de Doltigo Tinalco
Santa Domilla
Tziupab
Santiago Ixtlángo Díe
 
----
iter 34922, loss: 17.135759
----
 nvayus
Sanguilon
Santo Temomina
Santo Tutey
Santo Tóntepie
Santo Dochinteltepec
Santúm Setepec
Santo Mumalos Val AneSompa
Santeára
Santo las Selcosto
Santo Nomitlán
San Yuacongo
Santo Tumos Morosonogo 
----
iter 34960, loss: 17.147679
----
 che
Sazintepec
Sulindo
Sacha de Goroyuacis
Siterinco do Telmosuito
SaSonto Marté Topatán Rona
Sucara
Socán
Suiutác
SineSon Tlapa
Saya
Sokiltepec
Silonana
Solcuacis
Tutialtepin
Sayua delda Jeyalco Gait 
----
iter 34998, loss: 17.446884
----
 ultapitlán
Tanacánco
Salonacoh

----
 nahujue de Álaxho
Gunemápá
Doszerpaa
Emodama Juatlán
Genite Albrral
Gumuciacac
Guneradelpec
Galsiva
Gencuero
Geríate
Guadelli Zapetlání
Dirrema
Guartaco de Maledtepec
Genacá
Giarpa
Hartancaala de Bend 
----
iter 36214, loss: 20.866424
----
 a
Huixo
Yuuula
Huatla
Guélapenounat
Hurcho
Marayapo Huankizul
Huadallo
Estalalatla
Humadán
Huáretla Luisa
Hua
Huanouga de Hudañe
Huatla
Arinampianíl
Hoizuncilzoncuauataetidos
Huascapancébora
Huilo Him 
----
iter 36252, loss: 20.920921
----
 uez Magesches Radapa
Gullohurhuuca
Gaértorca
Hucán
Huechula
Atzobalcinga
Dokengo
Huiyanat
Huenlatla
Hoza,no
Huidasohua
Hiiés
Himende
Hiechin
Huihuimarpen
Huá
Huitlán
Ayutla de Bundel de Hertago
Hulild 
----
iter 36290, loss: 20.851106
----
 dana del Salidatla
Ixtzeraraaso
Imiloz
Ixhualotoral Herúndechela
Ixcautla
Huatla de Tumeme
Ixzejatos
Ixreapac
Mololcastepec
Ixtemonda
Ikúnche Tlares Z. Lumón
Ixcopín
Izarrislohual
Ixtlau Ixcayaca Hind 
----
iter 36328, loss: 20.910355
----
 a
Lala
Felonde
Jomra
I. datlán

----
 quila Larayenta Sanacan Aquáubas delán E pasta
San Nicas
Santo
Santepec
San Bedronecutlán Cumoso
San Salcán
San Lagdilitlán
Sancaco
Sant Gería
Santo Mendollán
San yocango Vigosamaa
Santuán Ixtlquiel T 
----
iter 37544, loss: 18.524426
----
 cama
Santa Cajanco
Santa Mantoya
Santa
Santa Valin Alartia Tachimán
Santa Crosán
Santa de Juárana
Santa Catín Alápas
Santa Catanca Quiabhicanape
Santea Aras Oe tepec
Santimiola
Santa Dtintlapaa
Santa  
----
iter 37582, loss: 18.412180
----
 l Franti
Santialal
San Jusima
Santa Crtzaxima
San Ytza Juárila Tida Plajis Antapan
Santo Crixo Moruna
Santa Crinta
Santa Crey Tentnal Quutipan CárCería Co Miasn
Santza
Santa Cutiran Nacay
Sando
Santa  
----
iter 37620, loss: 18.281577
----
 péco
Santa Maguela
Santa Crcala Daqueltla
Santa Isova Houtlán
Santa Cato
Santa Cralotepec
Santa Cpevez Chucua
Santa Lpblz
Santa Marachas Ata Miruxtla
Santa Cauz
Santa Marlabal Socnada
Santa Crima
Sant 
----
iter 37658, loss: 18.227915
----
 anta Mariarepen
Santa Mariíca


## Redes recurrentes tipo LSTM

Las redes con unidades LSTM las vimos platicadas en clase, pero no hay nada mejor para entender un tema que implementarlo y comparar los resultados con la red recurrente simple, sin memoria, para esto vamos a hacer lo mismo que antes, pero con unidades LSTM.

Para esto vamos a utilizar otro [gist](https://gist.github.com/karpathy/587454dc0146a6ae21fc) del mismo autor (que es una referencia obligada en el tema, por cierto). En este *gist*, el autor presenta un modelo de redes recurrentes LSTM desarrollado con *numpy* incluido el método de entrenamiento, pero no lo aplica a el modelado *letra a letra* como el gist pasado. Para esta parte de la libreta, lo que tienen que realizar es lo siguiente:


1. Copiar el contenido del *gist* y comentarlo en español (y cambiar algo de código de forma que quede mñas claro para ti y para mi).

2. Adaptar el modelo propuesto para usarlo en la generación de nombres de municipios.

3. Ajustar los hiperparámetros del modelo, así como los parámetros del algoritmo de entrenamiento con el fin de generar una lista de nombres de municipios creibles, pero sin sobreaprendizaje (esto es, que copie vilmente el nombre de municipios existentes). 


### Clase de LSTM

In [78]:
class LSTM:
    @staticmethod
    def init(input_size, hidden_size, fancy_forget_bias_init = 3):
        """ 
        Initialize parameters of the LSTM (both weights and biases in one matrix) 
        One might way to have a positive fancy_forget_bias_init number (e.g. maybe even up to 5, in some papers)
        """
        # +1 for the biases, which will be the first row of WLSTM
        WLSTM = np.random.randn(input_size + hidden_size + 1, 4 * hidden_size) / np.sqrt(input_size + hidden_size)
        WLSTM[0,:] = 0 # initialize biases to zero
        if fancy_forget_bias_init != 0:
            # forget gates get little bit negative bias initially to encourage them to be turned off
            # remember that due to Xavier initialization above, the raw output activations from gates before
            # nonlinearity are zero mean and on order of standard deviation ~1
            WLSTM[0,hidden_size:2*hidden_size] = fancy_forget_bias_init
        return WLSTM
  
    @staticmethod
    def forward(X, WLSTM, c0 = None, h0 = None):
        """
        X should be of shape (n,b,input_size), where n = length of sequence, b = batch size
        """
        n,b,input_size = X.shape
        d = int(WLSTM.shape[1]/4) # hidden size
        if c0 is None: c0 = np.zeros((b,d))
        if h0 is None: h0 = np.zeros((b,d))

        # Perform the LSTM forward pass with X as the input
        xphpb = WLSTM.shape[0] # x plus h plus bias, lol
        Hin = np.zeros((n, b, xphpb)) # input [1, xt, ht-1] to each tick of the LSTM
        Hout = np.zeros((n, b, d)) # hidden representation of the LSTM (gated cell content)
        IFOG = np.zeros((n, b, d * 4)) # input, forget, output, gate (IFOG)
        IFOGf = np.zeros((n, b, d * 4)) # after nonlinearity
        C = np.zeros((n, b, d)) # cell content
        Ct = np.zeros((n, b, d)) # tanh of cell content
        for t in range(n):
            # concat [x,h] as input to the LSTM
            prevh = Hout[t-1] if t > 0 else h0
            Hin[t,:,0] = 1 # bias
            Hin[t,:,1:input_size+1] = X[t]
            Hin[t,:,input_size+1:] = prevh
            # compute all gate activations. dots: (most work is this line)
            IFOG[t] = Hin[t].dot(WLSTM)
            # non-linearities
            IFOGf[t,:,:3*d] = 1.0/(1.0+np.exp(-IFOG[t,:,:3*d])) # sigmoids; these are the gates
            IFOGf[t,:,3*d:] = np.tanh(IFOG[t,:,3*d:]) # tanh
            # compute the cell activation
            prevc = C[t-1] if t > 0 else c0
            C[t] = IFOGf[t,:,:d] * IFOGf[t,:,3*d:] + IFOGf[t,:,d:2*d] * prevc
            Ct[t] = np.tanh(C[t])
            Hout[t] = IFOGf[t,:,2*d:3*d] * Ct[t]

        cache = {}
        cache['WLSTM'] = WLSTM
        cache['Hout'] = Hout
        cache['IFOGf'] = IFOGf
        cache['IFOG'] = IFOG
        cache['C'] = C
        cache['Ct'] = Ct
        cache['Hin'] = Hin
        cache['c0'] = c0
        cache['h0'] = h0

        # return C[t], as well so we can continue LSTM with prev state init if needed
        return Hout, C[t], Hout[t], cache

    @staticmethod
    def backward(dHout_in, cache, dcn = None, dhn = None): 

        WLSTM = cache['WLSTM']
        Hout = cache['Hout']
        IFOGf = cache['IFOGf']
        IFOG = cache['IFOG']
        C = cache['C']
        Ct = cache['Ct']
        Hin = cache['Hin']
        c0 = cache['c0']
        h0 = cache['h0']
        n,b,d = Hout.shape
        input_size = WLSTM.shape[0] - d - 1 # -1 due to bias

        # backprop the LSTM
        dIFOG = np.zeros(IFOG.shape)
        dIFOGf = np.zeros(IFOGf.shape)
        dWLSTM = np.zeros(WLSTM.shape)
        dHin = np.zeros(Hin.shape)
        dC = np.zeros(C.shape)
        dX = np.zeros((n,b,input_size))
        dh0 = np.zeros((b, d))
        dc0 = np.zeros((b, d))
        dHout = dHout_in.copy() # make a copy so we don't have any funny side effects
        if dcn is not None: dC[n-1] += dcn.copy() # carry over gradients from later
        if dhn is not None: dHout[n-1] += dhn.copy()
        for t in reversed(range(n)):

            tanhCt = Ct[t]
            dIFOGf[t,:,2*d:3*d] = tanhCt * dHout[t]
            # backprop tanh non-linearity first then continue backprop
            dC[t] += (1-tanhCt**2) * (IFOGf[t,:,2*d:3*d] * dHout[t])

            if t > 0:
                dIFOGf[t,:,d:2*d] = C[t-1] * dC[t]
                dC[t-1] += IFOGf[t,:,d:2*d] * dC[t]
            else:
                dIFOGf[t,:,d:2*d] = c0 * dC[t]
                dc0 = IFOGf[t,:,d:2*d] * dC[t]
                
            dIFOGf[t,:,:d] = IFOGf[t,:,3*d:] * dC[t]
            dIFOGf[t,:,3*d:] = IFOGf[t,:,:d] * dC[t]

            # backprop activation functions
            dIFOG[t,:,3*d:] = (1 - IFOGf[t,:,3*d:] ** 2) * dIFOGf[t,:,3*d:]
            y = IFOGf[t,:,:3*d]
            dIFOG[t,:,:3*d] = (y*(1.0-y)) * dIFOGf[t,:,:3*d]

            # backprop matrix multiply
            dWLSTM += np.dot(Hin[t].transpose(), dIFOG[t])
            dHin[t] = dIFOG[t].dot(WLSTM.transpose())

            # backprop the identity transforms into Hin
            dX[t] = dHin[t,:,1:input_size+1]
            if t > 0:
                dHout[t-1,:] += dHin[t,:,input_size+1:]
            else:
                dh0 += dHin[t,:,input_size+1:]

        return dX, dWLSTM, dc0, dh0

### verificar emparejamiento secuenciales en batches

In [77]:
def checkSequentialMatchesBatch():
    """ check LSTM I/O forward/backward interactions """

    n,b,d = (5, 3, 4) # sequence length, batch size, hidden size
    input_size = 10
    WLSTM = LSTM.init(input_size, d) # input size, hidden size
    X = np.random.randn(n,b,input_size)
    h0 = np.random.randn(b,d)
    c0 = np.random.randn(b,d)

    # sequential forward
    cprev = c0
    hprev = h0
    caches = [{} for t in range(n)]
    Hcat = np.zeros((n,b,d))
    for t in range(n):
        xt = X[t:t+1]
        _, cprev, hprev, cache = LSTM.forward(xt, WLSTM, cprev, hprev)
        caches[t] = cache
        Hcat[t] = hprev

    # sanity check: perform batch forward to check that we get the same thing
    H, _, _, batch_cache = LSTM.forward(X, WLSTM, c0, h0)
    assert np.allclose(H, Hcat), 'Sequential and Batch forward don''t match!'

    # eval loss
    wrand = np.random.randn(*Hcat.shape)
    loss = np.sum(Hcat * wrand)
    dH = wrand

    # get the batched version gradients
    BdX, BdWLSTM, Bdc0, Bdh0 = LSTM.backward(dH, batch_cache)

    # now perform sequential backward
    dX = np.zeros_like(X)
    dWLSTM = np.zeros_like(WLSTM)
    dc0 = np.zeros_like(c0)
    dh0 = np.zeros_like(h0)
    dcnext = None
    dhnext = None
    for t in reversed(range(n)):
        dht = dH[t].reshape(1, b, d)
        dx, dWLSTMt, dcprev, dhprev = LSTM.backward(dht, caches[t], dcnext, dhnext)
        dhnext = dhprev
        dcnext = dcprev

        dWLSTM += dWLSTMt # accumulate LSTM gradient
        dX[t] = dx[0]
        if t == 0:
            dc0 = dcprev
            dh0 = dhprev

    # and make sure the gradients match
    print('Making sure batched version agrees with sequential version: (should all be True)')
    print(np.allclose(BdX, dX))
    print(np.allclose(BdWLSTM, dWLSTM))
    print(np.allclose(Bdc0, dc0))
    print(np.allclose(Bdh0, dh0))

### Verificar gradiente en batches

In [79]:
def checkBatchGradient():
    """ check that the batch gradient is correct """

    # lets gradient check this beast
    n,b,d = (5, 3, 4) # sequence length, batch size, hidden size
    input_size = 10
    WLSTM = LSTM.init(input_size, d) # input size, hidden size
    X = np.random.randn(n,b,input_size)
    h0 = np.random.randn(b,d)
    c0 = np.random.randn(b,d)

    # batch forward backward
    H, Ct, Ht, cache = LSTM.forward(X, WLSTM, c0, h0)
    wrand = np.random.randn(*H.shape)
    loss = np.sum(H * wrand) # weighted sum is a nice hash to use I think
    dH = wrand
    dX, dWLSTM, dc0, dh0 = LSTM.backward(dH, cache)

    def fwd():
        h,_,_,_ = LSTM.forward(X, WLSTM, c0, h0)
        return np.sum(h * wrand)

    # now gradient check all
    delta = 1e-5
    rel_error_thr_warning = 1e-2
    rel_error_thr_error = 1
    tocheck = [X, WLSTM, c0, h0]
    grads_analytic = [dX, dWLSTM, dc0, dh0]
    names = ['X', 'WLSTM', 'c0', 'h0']
    for j in range(len(tocheck)):
        mat = tocheck[j]
        dmat = grads_analytic[j]
        name = names[j]
        # gradcheck
        for i in range(mat.size):
            old_val = mat.flat[i]
            mat.flat[i] = old_val + delta
            loss0 = fwd()
            mat.flat[i] = old_val - delta
            loss1 = fwd()
            mat.flat[i] = old_val

            grad_analytic = dmat.flat[i]
            grad_numerical = (loss0 - loss1) / (2 * delta)

            if grad_numerical == 0 and grad_analytic == 0:
                rel_error = 0 # both are zero, OK.
                status = 'OK'
            elif abs(grad_numerical) < 1e-7 and abs(grad_analytic) < 1e-7:
                rel_error = 0 # not enough precision to check this
                status = 'VAL SMALL WARNING'
            else:
                rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
                status = 'OK'
                if rel_error > rel_error_thr_warning: status = 'WARNING'
                if rel_error > rel_error_thr_error: status = '!!!!! NOTOK'

        # print stats
        print('%s checking param %s index %s (val = %+8f), analytic = %+8f, numerical = %+8f, relative error = %+8f' \
              % (status, name, 'np.unravel_index(i, mat.shape)', old_val, grad_analytic, grad_numerical, rel_error))

### Main

In [81]:
tol = 17

# hyperparameters
hidden_size = 38 # size of hidden layer of neurons
seq_length = 12 # number of steps to unroll the RNN for
learning_rate = 0.2

# lets gradient check this beast
    n,b,d = (5, hidden_size) # sequence length, batch size, hidden size
    input_size = 10
WLSTM = LSTM.init(input_size, d) # input size, hidden size

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

every line should start with OK. Have a nice day!


Por último, agrega en esta misma celda comentarios sobre la comparación entre los dos modelos vistos, diferencias, similitudes, ventajas, desventajas. Recuerda es una opinión personal basada en tu trabajo, no pongas lo que dice la literatura si no lo que tu experimentaste a la hora de desarrollar y aplicar los modelos, así no concuerde con lo que leas en otros lados.